In [31]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ahmdtolba","key":"fe5319345218adc8d6879e3c36bc1381"}'}

In [32]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [33]:
!kaggle datasets download -d crowdflower/first-gop-debate-twitter-sentiment

100% 2.40M/2.40M [00:00<00:00, 19.0MB/s]



In [34]:
from zipfile import ZipFile
file_name='/content/first-gop-debate-twitter-sentiment.zip'
with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [35]:
# Import Libraries 
import numpy as np 
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [36]:
# Read-Data
data = pd.read_csv('/content/Sentiment.csv')
data = data[['text','sentiment']]

In [37]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [38]:
data.sentiment.unique()

array(['Neutral', 'Positive', 'Negative'], dtype=object)

In [41]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print( "Positive Data : " , data[ data['sentiment'] == 'Positive'].size)
print("Negative Data : " , data[ data['sentiment'] == 'Negative'].size)
print("Neutral Data : "  , data[ data['sentiment'] == 'Neutral'].size)


Positive Data :  4472
Negative Data :  16986
Neutral Data :  6284


In [42]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [44]:
max_fatures = 3000
# Tokenizer Convert Comments to words
tokenizer = Tokenizer(num_words = max_fatures , split=' ')
tokenizer.fit_on_texts(data['text'].values) 
X = tokenizer.texts_to_sequences(data['text'].values)
# PAd_sequnce maxking all comment with the same size by adding 0 or shorting comment
X = pad_sequences(X)

In [45]:
embed_dim = 128
lstm_out = 196
no_classes = 3

model = Sequential()
model.add(Embedding(max_fatures, embed_dim , input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(no_classes,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 29, 128)           384000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 29, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 591       
Total params: 639,391
Trainable params: 639,391
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
Y = pd.get_dummies( data['sentiment'] ).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(" X_Train Shape is : " , X_train.shape , " : Y_tain Shape is :  ",Y_train.shape)
print(" X_Test Shape is : " , X_test.shape , " : Y_test Shape is : " , Y_test.shape)

 X_Train Shape is :  (9293, 29)  : Y_tain Shape is :   (9293, 3)
 X_Test Shape is :  (4578, 29)  : Y_test Shape is :  (4578, 3)


In [47]:
batch_size = 42

model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
222/222 - 31s - loss: 0.8365 - accuracy: 0.6425
Epoch 2/7
222/222 - 28s - loss: 0.6913 - accuracy: 0.7098
Epoch 3/7
222/222 - 29s - loss: 0.6103 - accuracy: 0.7507
Epoch 4/7
222/222 - 28s - loss: 0.5651 - accuracy: 0.7638
Epoch 5/7
222/222 - 30s - loss: 0.5171 - accuracy: 0.7898
Epoch 6/7
222/222 - 29s - loss: 0.4760 - accuracy: 0.8052
Epoch 7/7
222/222 - 30s - loss: 0.4439 - accuracy: 0.8194


In [48]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

74/74 - 1s - loss: 0.8855 - accuracy: 0.6676
score: 0.89
acc: 0.67


In [64]:
text = ['i will kill you']
#vectorizing the tweet by the pre-fitted tokenizer instance
text = tokenizer.texts_to_sequences(text)
#padding the tweet to have exactly the same shape as `embedding_2` input
text = pad_sequences(text, maxlen=28, dtype='int32', value=0)
print(text)
sentiment = model.predict(text,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")
else :
  print("neural")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   9  70 276  12]]
1/1 - 0s
negative
